# Building Conversational GenAI Chatbots for Enterprises - Workshop
---



This notebook is based on this aws blog:
* https://aws.amazon.com/blogs/machine-learning/quickly-build-high-accuracy-generative-ai-applications-on-enterprise-data-using-amazon-kendra-langchain-and-large-language-models/

And this repo:
* https://github.com/aws-samples/amazon-kendra-langchain-extensions

---

### Solution overview
The following diagram shows the architecture of a GenAI application with a RAG approach.

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2023/05/02/ML-13807-image001-new.png">

We use an Amazon Kendra index to ingest enterprise unstructured data from data sources such as wiki pages, MS SharePoint sites, Atlassian Confluence, and document repositories such as Amazon S3. When a user interacts with the GenAI app, the flow is as follows:

1. The user makes a request to the GenAI app.
2. The app issues a search query to the Amazon Kendra index based on the user request.
3. The index returns search results with excerpts of relevant documents from the ingested enterprise data.
4. The app sends the user request and along with the data retrieved from the index as context in the LLM prompt.
5. The LLM returns a succinct response to the user request based on the retrieved data.
6. The response from the LLM is sent back to the user.

With this architecture, you can choose the most suitable LLM for your use case. LLM options include our partners Meta, Hugging Face, AI21 Labs, Cohere, and others hosted on an Amazon SageMaker endpoint, as well as models by companies like Anthropic and OpenAI. With Amazon Bedrock, you will be able to choose Amazon Titan, Amazon’s own LLM, or partner LLMs such as those from AI21 Labs and Anthropic with APIs securely without the need for your data to leave the AWS ecosystem. The additional benefits that Amazon Bedrock will offer include a serverless architecture, a single API to call the supported LLMs, and a managed service to streamline the developer workflow.

For the best results, a GenAI app needs to engineer the prompt based on the user request and the specific LLM being used. Conversational AI apps also need to manage the chat history and the context. GenAI app developers can use open-source frameworks such as LangChain that provide modules to integrate with the LLM of choice, and orchestration tools for activities such as chat history management and prompt engineering. We have provided the KendraIndexRetriever class, which implements a LangChain retriever interface, which applications can use in conjunction with other LangChain interfaces such as chains to retrieve data from an Amazon Kendra index. We have also provided a few sample applications in the GitHub repo. You can deploy this solution in your AWS account using the step-by-step guide in this post.

---

0. [Prerequisites](#Prerequisites)
1. [Permissions and environment variables](#1.-Permissions-and-environment-variables)
2. [Select a pre-trained model](#2.-Select-a-pre-trained-model)
3. [Retrieve Artifacts & Deploy an Endpoint](#3.-Retrieve-Artifacts-&-Deploy-an-Endpoint)
4. [Query endpoint and parse response](#4.-Query-endpoint-and-parse-response)
5. [Query endpoint with Langchain and Kendra Index](#5.-Query-endpoint-with-Langchain-and-Kendra-Index)
6. [[OPTIONAL] Installing Streamlet application and running a WebUI for a chatbot](#6.-[OPTIONAL]-Installing-Streamlit-application-and-running-a-WebUI-for-a-chatbot)
7. [Clean up the endpoint](#7.-Clean-up-the-endpoint)

---

# Prerequisites

# Installation and import of required dependencies, further setup tasks

For this lab, we will use the following libraries:

 - SageMaker SDK for interacting with Amazon SageMaker
 - boto3, the AWS SDK for python
 - os, a python library implementing miscellaneous operating system interfaces 
 - tarfile, a python library to read and write tar archive files
 - io, native Python library, provides Python’s main facilities for dealing with various types of I/O.
 - tqdm, a utility to easily show a smart progress meter for synchronous operations.

In [ ]:
import sagemaker
import boto3
import os
import tarfile
import requests
from io import BytesIO
from tqdm import tqdm

## if you get an error message about the NumPy and SciPy version you can safely ignore it and move on

# Setup of notebook environment

Before we begin with the actual work, we need to setup the notebook environment respectively. This includes:

- retrieval of the execution role our SageMaker Studio domain is associated with for later usage
- retrieval of our account_id for later usage
- retrieval of the chosen region for later usage

In [ ]:
# Retrieve SM execution role
aws_role = sagemaker.get_execution_role()

In [ ]:
# Create a new STS client
sts_client = boto3.client('sts')

# Call the GetCallerIdentity operation to retrieve the account ID
response = sts_client.get_caller_identity()
account_id = response['Account']
account_id

In [ ]:
# Retrieve region
aws_region = boto3.Session().region_name
aws_region

# Setup of S3 bucket for Amazon Kendra storage of knowledge documents

Amazon Kendra provides multiple built-in adapters for integrating with data sources to build up a document index, e.g. S3, Microsoft SharePoint sites, Atlassian Confluence, web-scraper, RDS, Box, Dropbox, ... . In this lab we will store the documents containing the Enterprise data knowledge to be infused into the application in S3. For this purpose we will create a dedicated S3 bucket.

## Uploading knowledge documents into an Amazon Kendra index

Next we are going to add some more documents from S3 to show how easy it is to integrate different data sources to a Kendra Index. 
First we are going to download some interesting pdf files from the internet, but please feel free to drop any pdf you might find interesting in it as well. 
In our case we will add some financial document relating to Amazon's yearly performance, also some client document for banking, and lastly some AWS Documentation docs.

Important: Before you proceed to the create the bucket in the next step, delete the Kendra Data Source named 'genAI_conf_Kendra_Data_Source' (this is provisioned as part of the Studio Engine but is used in a different workshop context and not relevant for ours). 

To Delete it:

1) Navigate to the Kendra console: https://us-east-1.console.aws.amazon.com/kendra/home?region=us-east-1#indexes
2) Click on the Only Kendra Index (starts with "genai-conference-index-xxxxx") 
3) On the left side of the console, under 'Data Management', select 'Data sources' 
4) Select the only Data source (named 'genAI_conf_Kendra_Data_Source') 
5) Select 'Actions' and click 'Delete' to delete this Data source (type 'Delete' in box to confirm)
6) You'd get a notification that the Data Source is being deleted, please proceed with the next step while this is happening in the background.

<img src="https://github.com/senatoredu/genai-kendra-rag/blob/main/datasourcedelete.png?raw=true">

Proceed to run the next cell to create your S3 bucket to store the Files and later create your own S3 Data Connector pointing to this bucket.

In [ ]:
import os
import boto3
import requests
from io import BytesIO
from tqdm import tqdm

# Create an S3 client
s3 = boto3.client('s3')

# Create a bucket if it doesn't exist
bucket_name = f'immersion-day-bucket-{account_id}-{aws_region}'
if s3.list_buckets()['Buckets']:
    for bucket in s3.list_buckets()['Buckets']:
        if bucket['Name'] == bucket_name:
            break
    else:
        s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(Bucket=bucket_name)

# List of URLs to download PDFs from
pdf_urls = [
    "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Annual-Report.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Annual-Report.pdf", 
    "https://s2.q4cdn.com/299287126/files/doc_financials/annual/2018-Annual-Report.pdf",
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/microservices-on-aws/microservices-on-aws.pdf",
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/overview-aws-cloud-adoption-framework/overview-aws-cloud-adoption-framework.pdf",
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/aws-multi-region-fundamentals/aws-multi-region-fundamentals.pdf",
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/aws-overview/aws-overview.pdf",
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/docker-on-aws/docker-on-aws.pdf", 
    "https://docs.aws.amazon.com/pdfs/whitepapers/latest/overview-deployment-options/overview-deployment-options.pdf",
    "https://www.nab.com.au/content/dam/nabrwd/documents/terms-and-conditions/banking/nab-personal-transactions-savings-terms-and-conditions.pdf",
    "https://www.nab.com.au/content/dam/nabrwd/documents/terms-and-conditions/banking/personal-transaction-and-savings-offsale.pdf",
    "https://budget.gov.au/content/bp1/download/bp1_2023-24_230727.pdf", 
    "https://budget.gov.au/content/overview/download/budget_overview-20230511.pdf",
    "https://budget.gov.au/content/bp2/download/bp2_2023-24.pdf",
    "https://archive.budget.gov.au/2022-23/bp2/download/bp2_2022-23.pdf",
]

# Download PDFs from the URLs and upload them to the S3 bucket
for url in tqdm(pdf_urls):
    response = requests.get(url, stream=True)
    filename = os.path.basename(url)
    print(f"Working on {filename}")
    fileobj = BytesIO()
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        fileobj.write(data)
    progress_bar.close()
    fileobj.seek(0)
    s3.upload_fileobj(fileobj, bucket_name, filename)

Lets use those documents in Kendra. First navigate to the Kendra console. 

Under "Data Management" you will find the tab "Data Sources". Navigate there and add a new data source via "Add data source". 
Take some time to inspect all the different connectors that are there for you to use out of the box. We will use s3 as our source. 

It is worth noting that Kendra respect enterprise level access attributes. That means, that it can deny queries if a user is not authorized to retrieve a document. 

The animation below shows how to add an s3 data source to kendra to index. We are creating a new IAM role as well as setting the indexing frequncy to "on-demand". 

<p align="center">
  <img src="https://raw.githubusercontent.com/aws-samples/generative-ai-on-aws-immersion-day/4727dd546aa15eeef0440aa53a39ecf85aa49e17/img/new_s3_connection.gif" alt="How to add a kendra s3 data source "/>
</p>


After the connection has been established, you can sync your data source by clicking "sync now". 

### 1. Permissions and environment variables for SageMaker

---
To host the LLaMA model on Amazon SageMaker, we need to set up and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook as the AWS account role with SageMaker access. 

---

In [ ]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

### 2. Select a pre-trained model
***
You can continue with the default model, or can choose a different model. A complete list of SageMaker pre-trained models can also be accessed at [SageMaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#). Be sure to select a model that can be used for text2text generation.

For our workshop we will use Llama 7b, which is a Llama 2 model from Meta that contains 7 billion parameters and is optimized for dialogue and assistant-like use cases. Llama 2 was pre-trained on 2 trillion tokens of data from publicly available sources. 
More details about its use and technical details here: 

https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/
***

In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "*"

### 3. Retrieve Artifacts & Deploy an Endpoint

***

Using SageMaker, we can perform inference on the pre-trained model, even without fine-tuning it first on a new dataset. We start by retrieving the `deploy_image_uri`, `deploy_source_uri`, and `model_uri` for the pre-trained model. To host the pre-trained model, we create an instance of [`sagemaker.model.Model`](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) and deploy it. This may take a few minutes.

***

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel
my_model = JumpStartModel(model_id = "meta-textgeneration-llama-2-7b-f")
predictor = my_model.deploy()

Once SageMaker is done deploying the model on an endpoint you'd see a '----------------!' output. 
You can also see the SageMaker model and endpoint by navigating to the 'SageMaker Dashboard' tab on the SageMaker console.

### 6. Installing Streamlit application and running a WebUI for a chatbot

---
This sections provides instructions on how to run a streamlit application within sagemaker studio and accessing it using jupyter proxy. The commands and instructions below need to be run inside a **SageMaker System Terminal**. 

First we will download a repo that contains a set of samples to work with Langchain and Amazon Kendra. It currently has samples for working with a Kendra retriever class to execute a QA chain for Meta, Open AI and Anthropic providers. From the repo we will install all the required modules and dependencies we need.

---

1. Launch a new SageMaker System Terminal 
   1. From the SageMaker Studio Home screen select `Open Launcher`
   2. From the Launcher panel under `Utilities and files` select `System terminal`
   
Clone the repository
```bash
git clone https://github.com/senatoredu/amazon-kendra-langchain-extensions.git
```

Move to the repo dir
```bash
cd amazon-kendra-langchain-extensions
```

Move to the samples dir
```bash
cd kendra_retriever_samples
```

Install the dependencies using pip (if you get a pip dependency resolver error at end safely ignore)


```bash
pip install -r requirements.txt
```


2. Activate the conda environment
```
conda activate studio
```

3. Set your environment variables. See image below if you need help finding the ID and Endpoint.
```
export AWS_REGION=us-east-1
export KENDRA_INDEX_ID="<YOUR_KENDRA_INDEX_ID>"
export LLAMA_2_ENDPOINT="<YOUR_SAGEMAKER_ENDPOINT_FOR_LLAMA_2>"

```
<p align="center">
    <img src="https://github.com/senatoredu/genai-kendra-rag/blob/main/kendraindex.png?raw=true"> 
</p> 

<p align="center">
    <img src="https://github.com/senatoredu/genai-kendra-rag/blob/main/sagemakerendpoint.png?raw=true">
</p> 

<p align="center">
    <img src="https://github.com/senatoredu/genai-kendra-rag/blob/main/sagemaker_endpointname.png?raw=true">
</p> 


4. Run the streamlit application
```
cd /home/sagemaker-user/amazon-kendra-langchain-extensions/kendra_retriever_samples/

streamlit run app.py llama2
```
5. This will output something similar to the below.
```
Collecting usage statistics. To deactivate, set browser.gatherUsageStats to False.


  You can now view your Streamlit app in your browser.

  Network URL: http://169.255.255.2:8501
  External URL: http://18.213.200.192:8501
```
6. Copy the current URL of the SageMaker Studio which should have the form (you can copy the URL from your current browser tab of sagemaker studio):
```
https://<YOUR_STUDIO_DOMAIN>.studio.<AWS_REGION>.sagemaker.aws/jupyter/default/lab/workspaces/auto-Z/tree/kendra_rag_demo.ipynb
```
7. Delete everything from `lab/` onwards and replace it with `proxy/<PORT>/`
   1. DON'T FORGET THE END `/`
```
https://<YOUR_STUDIO_DOMAIN>.studio.<AWS_REGION>.sagemaker.aws/jupyter/default/proxy/8501/
```
8. Paste the new address into the browser and you will now be able to access your chatbot UI which uses Langchain and Kendra. Each response will list the sources from Kendra it used for its answers.

### 7. Clean up the endpoint
---
When you're done with the lab and if you'd like to delete the SageMaker endpoint run the cell below.

In [ ]:
# Delete the SageMaker endpoint
model_predictor.delete_model()
model_predictor.delete_endpoint()